In [6]:
import pandas as pd
import numpy as np
import pickle

# Read Actual Bounding Boxes

In [2]:
file = open('../Data/Source_Images/Test_Images/data_test.txt', 'r')
file_names = []
xmin = []
ymin = []
xmax = []
ymax = []

line = file.readline()
while line:
    line_split = line.split()
    
    boxes = line_split[1:]
    for i in range(0, len(boxes)):
        boxes[i] = list(map(int, boxes[i].split(',')))
        file_names.append(line_split[0].split('/')[-1]) 
        xmin.append(boxes[i][0])
        ymin.append(boxes[i][1])
        xmax.append(boxes[i][2])
        ymax.append(boxes[i][3])

    line = file.readline()        
file.close()

actual = pd.DataFrame({'image': file_names,
                       'xmin': xmin,
                       'ymin': ymin,
                       'xmax': xmax,
                       'ymax': ymax})
actual

,image,xmin,ymin,xmax,ymax
0,0583a114dcce0256.jpg,403,158,503,257
1,0583b88307ee9826.jpg,116,268,156,308
2,0583b88307ee9826.jpg,750,193,796,263
3,0583b88307ee9826.jpg,938,177,998,250
4,0584dedb58d95f5e.jpg,273,479,331,533
...,...,...,...,...,...
982,083da45e6c56b9b0.jpg,335,71,663,522
983,083dc49c9b06ee05.jpg,182,314,558,662
984,083e8094d9eb4a86.jpg,120,410,143,447
985,083e8094d9eb4a86.jpg,463,373,486,410


# Read Predicted Bounding Boxes

In [3]:
predicted = pd.read_csv('../Data/Source_Images/Test_Image_Detection_Results/Detection_Results.csv')
predicted

,image,image_path,xmin,ymin,xmax,ymax,label,confidence,x_size,y_size
0,0583a114dcce0256.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,408,142,503,263,0,0.854823,1024,732
1,0583b88307ee9826.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,621,94,673,163,0,0.432138,1024,681
2,0583b88307ee9826.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,749,190,801,261,0,0.476733,1024,681
3,0583b88307ee9826.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,129,275,155,311,0,0.516130,1024,681
4,0583b88307ee9826.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,34,275,98,347,0,0.570330,1024,681
...,...,...,...,...,...,...,...,...,...,...
894,083e8094d9eb4a86.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,373,391,400,425,0,0.410875,681,1024
895,083e8094d9eb4a86.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,462,371,482,400,0,0.466528,681,1024
896,083e8094d9eb4a86.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,213,80,361,221,0,0.499436,681,1024
897,083e8094d9eb4a86.jpg,C:\Users\Simone\Desktop\COV\Projekt\TrainYourO...,513,318,547,359,0,0.814241,681,1024


# Evaluate (Intersection Over Union)

In [4]:
def bb_intersection_over_union(boxesA, boxesB):
    ious = []
    for boxA in boxesA.iterrows():
        boxA_ious = []
        boxA = boxA[1]
        
        for boxB in boxesB.iterrows():
            boxB = boxB[1]

            xA = max(boxA['xmin'], boxB['xmin'])
            yA = max(boxA['ymin'], boxB['ymin'])
            xB = min(boxA['xmax'], boxB['xmax'])
            yB = min(boxA['ymax'], boxB['ymax'])

            inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
            
            # compute the area of both the prediction and ground-truth
            # rectangles
            boxA_area = (boxA['xmax'] - boxA['xmin'] + 1) * (boxA['ymax'] - boxA['ymin'] + 1)
            boxB_area = (boxB['xmax'] - boxB['xmin'] + 1) * (boxB['ymax'] - boxB['ymin'] + 1)
            
            # compute the intersection over union by taking the intersection
            # area and dividing it by the sum of prediction + ground-truth
            # areas - the interesection area
            boxA_ious.append(inter_area / float(boxA_area + boxB_area - inter_area))
        
        if (len(boxA_ious) == 0):
            ious.append(0)
        else:
            ious.append(max(boxA_ious))
    
    return np.array(ious)        

In [5]:
ious = np.array([])
for file in actual['image'].unique():
    iou = bb_intersection_over_union(actual[actual['image'] == file], predicted[predicted['image'] == file])
    ious = np.concatenate((ious, iou), axis=None)

# mean iou
print(np.mean(ious))

# zero intersection ratio
zero_ratio = (ious == 0).sum() / ious.shape[0]
print(zero_ratio)

# non zero ratio
print(1 - zero_ratio)

0.41691167430958903
0.41033434650455924
0.5896656534954408


In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [9]:
with open('../Data/Model_Weights/history_stage_1.pkl', 'rb') as history_file:
    history = pickle.load(history_file)
plot_history(history)

test


In [ ]:
with open('../Data/Model_Weights/history_final.pkl', 'rb') as history_file:
    history = pickle.load(history_file)
plot_history(history)